In [1]:
using Pkg
Pkg.activate("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\env\\integrate\\")

using StaticArrays, DifferentialEquations, DynamicalSystems, JLD

  Activating project at `C:\Users\Alex\Desktop\repo\2612\0903\dynamical-systems\env\integrate`


In [2]:
@inbounds U(y, p) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
@inbounds σ(x, p) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
@inbounds g(E, x, y, p, U_) = log( 1.0 + exp( (p[5] * U_ * x * E + p[11]  ) / (p[1]) ) )

@inbounds function TM(u, p, t)
    
    U_ = U(u[3], p)
    
    du1 = (-u[1] + p[1] * g(u[1], u[2], u[3], p, U_) ) / p[2]
    du2 = (1.0 - u[2]) / p[3] - U_*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2], p)
    
    return SVector(du1, du2, du3)
end
@inbounds function jacob_TM_(u, p, t)
    
    U(y, p, exp50) = p[8] + p[9] / ( 1.0 + exp50 )
    U_y(y, p, exp50) = (50.0 * p[9] * exp50) / (1.0 + exp50)^2
    g(E, x, y, p, U_) = exp((p[5]  * U_ * x * E + p[11]) / p[1])
    σ_der(x, p) = exp( (-20.0) * (x - p[6]) )
    exp50 = exp(-50.0 * (u[3] - p[7]))
    
    U_ = U(u[3], p, exp50)
    Uy = U_y(u[3], p, exp50)
    g_ = g(u[1], u[2], u[3], p, U_)
    σ_deri = σ_der(u[2], p)
    
    g_plus = 1.0 + g_
    g_mult = g_ * U_
    g_plus_mult = p[2] * (g_plus)
    u1p5 = p[5] * u[1]
    Uyu2 = Uy * u[2]
    
    E_E = (-1.0 + ((J * u[2] * g_mult)) / (g_plus) ) / p[2]
    E_x = (u1p5 * g_mult) / (g_plus_mult)
    E_y = (u1p5 * Uyu2 * g_) / (g_plus_mult)
    
    x_E = -U_ * u[2]
    x_x = -1.0 / p[3] - U_ * u[1]
    x_y = -Uyu2 * u[1]
    
    y_x = 20.0 * p[10] * σ_deri / (1.0 + σ_deri)^2
    y_y = -1.0/p[4]
    
    SMatrix{3,3}(E_E, x_E, 0.0,
        E_x, x_x, y_x,
        E_y, x_y, y_y)
end;

In [3]:
t = 500; tstep = 0.001
trange = range(0.0, t, step = tstep)
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [4]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const ΔU0 = 0.305;

In [5]:
I0range = range( -0.5, -2.0,  length = 200 )
U0range = range( 0.22,  0.55, length = 200 )

0.22:0.0016582914572864321:0.55

In [6]:
path = "C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\";
filenameLS = "LSs_200_200_time_1000.jld"
filenameu0s = "u0s_200_200_time_1000.jld"
pathLS = path*filenameLS
pathu0s = path*filenameu0s;

In [7]:
Λs = load(pathLS)["data"]
u0s = load(pathu0s)["data"];

In [8]:
modes = zeros(200, 200)
filename = "Modes_200x200.jld"

"Modes_200x200.jld"

In [9]:
E = 0..30; x = 0..1; y = 0..1
box = E × x × y;

In [10]:
function check_fp(idx_I0, idx_U0)
    
    fp = 0
    I0_ = I0range[idx_I0]
    U0_ = U0range[idx_U0]
    p = [α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0_]
    
    ds = CoupledODEs(TM, u0s[idx_I0, idx_U0, :], p, diffeq = integ_set)
    _, eigs, _ = fixedpoints(ds, box, jacob_TM_)
    
    for (_, eig) in enumerate(eigs)
        
        Re = sort( abs.(real(eig)) )
        spectrum = abs.(Λs[idx_I0, idx_U0, :])

        if abs.(abs.(spectrum - Re)) <= [0.01, 0.01, 0.01]
            fp = -1
            return fp
        end
            
    end
    return fp
        
end

check_fp (generic function with 1 method)

In [11]:
function check_chaos_mode(idx_I0, idx_U0)
    chaos = 0
    if Λs[idx_I0, idx_U0, 1] >= 0.02
            chaos = 3
    end
    return chaos
end
function local_min(tr)
    
    localmin = Float64[]
    placeintime = Float64[]
    
    for idx in range(2, length(tr) - 1, step = 1)
        if tr[idx-1] > tr[idx] < tr[idx+1]
            push!(localmin, tr[idx])
            push!(placeintime, trange[idx])
        end
    end
    return localmin, placeintime
end

function traj(idx_I0, idx_U0)
    
    E, x, y = u0s[idx_I0, idx_U0, :]
    u0 = [E, x, y]
    I0_ = I0range[idx_I0]
    U0_ = U0range[idx_U0]
    
    p = [α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0_]
    
    ds = CoupledODEs(TM, u0, p, diffeq = integ_set)
    
    tr, _ = trajectory(ds, t, Δt = tstep)
    
    return tr[:, 1]
end

function detect_mode(idx_I0, idx_U0)
    
    mode = check_chaos_mode(idx_I0, idx_U0)
    if mode == 0
        
        
        fp = check_fp(idx_I0, idx_U0)

        if fp == -1
            return fp
        end
        
        tr = traj(idx_I0, idx_U0)
        lcmin, _ = local_min(tr)
        
        if abs(maximum(lcmin) - minimum(lcmin)) > 0.4 # Было 2 !
            mode = 2
        else
            mode = 1
        end
    end
    return mode
end

detect_mode (generic function with 1 method)

In [12]:
function output(idx_U0, U0, idx_I0, I0, mode)
    println("index U0: $idx_U0; index I0: $idx_I0");flush(stdout)
    println("U0: $U0");flush(stdout)
    println("I0: $I0");flush(stdout)
    println("Mode: $mode");flush(stdout)
end
function separate()
    println("");flush(stdout)
    println("----------------");flush(stdout)
end

separate (generic function with 1 method)

In [ ]:
for (idx_U0, U0) in enumerate(U0range)
    for (idx_I0, I0) in enumerate(I0range)
        
        modes[idx_I0,idx_U0] =  detect_mode(idx_I0, idx_U0)
        #output(idx_U0, U0, idx_I0, I0, modes[idx_I0,idx_U0])
        #separate()
        
    end
    save(filename, "data", modes)
end   